In [1]:
import pandas as pd
import numpy as np
from keras.models import Model, load_model
from keras.layers import LSTM, Dense, Dropout, Input, Conv1D, Flatten, concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import os
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Definir look_back
look_back = 6  

# Cargar el archivo y eliminar filas donde el clúster es NaN
file_path = 'data/data.csv'   # Cambia esta ruta al archivo que has subido
data = pd.read_csv(file_path)
data = data.dropna(subset=['Cluster_Label'])

# Convertir 'year' y 'month_no' a una sola columna de tipo fecha
data['date'] = pd.to_datetime(data['year'].astype(str) + '-' + data['month_no'].astype(str))

# Seleccionar las columnas necesarias para la serie temporal
columns_to_keep = ['date', 'species', 'Cluster_Label', 'landed_w_kg',
                   'mean_temp_30m','mean_temp_10m','thetao_sfc=6',
                   'thetao_sfc=7.92956018447876','thetao_sfc=9.572997093200684','thetao_sfc=11.40499973297119',
                   'thetao_sfc=13.46714019775391','thetao_sfc=15.8100700378418','thetao_sfc=18.49555969238281',
                   'thetao_sfc=21.59881973266602','thetao_sfc=25.21141052246094','thetao_sfc=29.44473075866699']
data = data[columns_to_keep]


2024-08-08 05:55:12.175709: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-08 05:55:12.178845: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-08 05:55:12.219428: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 05:55:12.219457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 05:55:12.220582: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Función para preparar los datos
def prepare_data(group, look_back=6):
    features = ['landed_w_kg','Cluster_Label', 'mean_temp_30m','mean_temp_10m','thetao_sfc=6',
                   'thetao_sfc=7.92956018447876','thetao_sfc=9.572997093200684','thetao_sfc=11.40499973297119',
                   'thetao_sfc=13.46714019775391','thetao_sfc=15.8100700378418','thetao_sfc=18.49555969238281',
                   'thetao_sfc=21.59881973266602','thetao_sfc=25.21141052246094','thetao_sfc=29.44473075866699']
    scaler = MinMaxScaler(feature_range=(0, 1))
    group_scaled = scaler.fit_transform(group[features])
    
    X, y = [], []
    for i in range(len(group_scaled) - look_back):
        a = group_scaled[i:(i + look_back)]
        X.append(a)
        y.append(group_scaled[i + look_back, 0])  # La primera columna es 'landed_w_kg'
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    return X, y, scaler

# Crear el modelo LSTM
def create_lstm_model(input_shape):
    lstm_input = Input(shape=input_shape)
    x = LSTM(50, return_sequences=True)(lstm_input)
    x = Dropout(0.2)(x)
    x = LSTM(25, activation='relu', return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(12, activation='linear')(x)
    lstm_output = Dense(1, activation='linear')(x)
    return Model(inputs=lstm_input, outputs=lstm_output)

# Crear el modelo DNN
def create_dnn_model(input_shape):
    dnn_input = Input(shape=(input_shape[0], input_shape[1]))
    x = Flatten()(dnn_input)
    x = Dense(10, activation='relu')(x)
    x = Dense(5, activation='relu')(x)
    dnn_output = Dense(1, activation='linear')(x)
    return Model(inputs=dnn_input, outputs=dnn_output)

# Crear el modelo CNN
def create_cnn_model(input_shape):
    cnn_input = Input(shape=input_shape)
    x = Conv1D(32, kernel_size=3, activation='relu')(cnn_input)
    x = Flatten()(x)
    cnn_output = Dense(1, activation='linear')(x)
    return Model(inputs=cnn_input, outputs=cnn_output)

# Crear el modelo Mixture of Experts (MoE)
def create_moe_model(input_shape):
    # Definir los expertos
    lstm_model = create_lstm_model(input_shape)
    dnn_model = create_dnn_model(input_shape)
    cnn_model = create_cnn_model(input_shape)
    
    # Definir el gating network
    moe_input = Input(shape=input_shape)
    x = Flatten()(moe_input)
    x = Dense(10, activation='relu')(x)
    gate_output = Dense(3, activation='softmax')(x)  # Tres expertos

    # Obtener las salidas de los expertos
    lstm_output = lstm_model(moe_input)
    dnn_output = dnn_model(moe_input)
    cnn_output = cnn_model(moe_input)
    
    # Combinar las salidas usando el gating network
    output = concatenate([lstm_output * gate_output[:, 0:1],
                          dnn_output * gate_output[:, 1:2],
                          cnn_output * gate_output[:, 2:3]], axis=1)
    output = Dense(1, activation='linear')(output)
    
    model = Model(inputs=moe_input, outputs=output)
    model.compile(optimizer=Adam(), loss='mean_squared_error')
    return model


In [3]:
# Función para entrenar y guardar el modelo MoE
def train_and_save_moe_model(data, species_name, cluster_label, look_back=look_back, epochs=50, batch_size=1):
    X, y, scaler = prepare_data(data, look_back)
    model = create_moe_model((look_back, X.shape[2]))
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    history = model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])
    
    # Guardar el modelo y el scaler
    model_directory = 'modelos_moe'
    os.makedirs(model_directory, exist_ok=True)
    model_path = os.path.join(model_directory, f'{species_name}_cluster_{cluster_label}_moe_model.h5')
    scaler_path = os.path.join(model_directory, f'{species_name}_cluster_{cluster_label}_moe_scaler.pkl')
    model.save(model_path)
    joblib.dump(scaler, scaler_path)
    
    # Guardar los valores de monitoreo del entrenamiento
    train_moe_directory = 'train_moe'
    os.makedirs(train_moe_directory, exist_ok=True)
    history_path = os.path.join(train_moe_directory, f'{species_name}_cluster_{cluster_label}_training_history.csv')
    pd.DataFrame(history.history).to_csv(history_path, index=False)
    
    print(f'Modelo MoE, scaler y datos de entrenamiento guardados para {species_name} en clúster {cluster_label}')
    return model, scaler

# Función para hacer predicciones con bandas de confianza usando bootstrap
def predict_with_confidence_intervals(model, scaler, data, look_back=6, n_bootstrap=100, alpha=0.01):
    features = ['landed_w_kg','Cluster_Label','mean_temp_30m','mean_temp_10m','thetao_sfc=6',
                   'thetao_sfc=7.92956018447876','thetao_sfc=9.572997093200684','thetao_sfc=11.40499973297119',
                   'thetao_sfc=13.46714019775391','thetao_sfc=15.8100700378418','thetao_sfc=18.49555969238281',
                   'thetao_sfc=21.59881973266602','thetao_sfc=25.21141052246094','thetao_sfc=29.44473075866699']
    group_scaled = scaler.transform(data[features])

    X = []
    for i in range(len(group_scaled) - look_back):
        a = group_scaled[i:(i + look_back)]
        X.append(a)
    X = np.array(X)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    
    predictions = model.predict(X)
    predictions = scaler.inverse_transform(np.hstack((predictions, X[:, -1, 1:])))[:, 0]
    
    # Bootstrap para bandas de confianza
    bootstrap_predictions = []
    for _ in range(n_bootstrap):
        indices = np.random.choice(range(len(X)), len(X), replace=True)
        X_sample = X[indices]
        pred_sample = model.predict(X_sample)
        pred_sample = scaler.inverse_transform(np.hstack((pred_sample, X_sample[:, -1, 1:])))[:, 0]
        bootstrap_predictions.append(pred_sample)
    
    bootstrap_predictions = np.array(bootstrap_predictions)
    lower_bound = np.percentile(bootstrap_predictions, 100 * alpha / 2, axis=0)
    upper_bound = np.percentile(bootstrap_predictions, 100 * (1 - alpha / 2), axis=0)
    
    return predictions, lower_bound, upper_bound



In [4]:
# Definir la ruta del directorio donde se almacenan los modelos
models_directory = 'modelos_moe'  # Cambia esto a la ruta real de tu directorio de modelos

# Filtrar las especies y clústeres únicos de la tabla de datos
unique_species = data['species'].unique()
unique_clusters = data['Cluster_Label'].unique()

# Crear una lista de los modelos esperados solo para combinaciones que tienen datos suficientes
expected_models = []
for species in unique_species:
    for cluster in unique_clusters:
        # Filtrar los datos para la combinación específica de especie y clúster
        filtered_data = data[(data['species'] == species) & (data['Cluster_Label'] == cluster)]
        # Verificar si hay suficientes datos para entrenar
        if len(filtered_data) > 10:
            expected_models.append(f'{species}_cluster_{cluster}_moe_model.h5')

# Obtener la lista de archivos existentes en el directorio de modelos
existing_models = os.listdir(models_directory)

# Identificar los modelos faltantes
missing_models = [model for model in expected_models if model not in existing_models]



In [5]:
missing_models

['MOJARRA_cluster_4_moe_model.h5',
 'CORVINA_cluster_2_moe_model.h5',
 'PARGO_cluster_2_moe_model.h5',
 'SIERRA_cluster_2_moe_model.h5',
 'JUREL_cluster_2_moe_model.h5',
 'GUACHINANGO_cluster_2_moe_model.h5',
 'GUACHINANGO_cluster_4_moe_model.h5',
 'BAQUETA_cluster_2_moe_model.h5',
 'LENGUADO_cluster_2_moe_model.h5',
 'CABRILLA_cluster_2_moe_model.h5',
 'LISA_cluster_2_moe_model.h5',
 'ALMEJA_cluster_2_moe_model.h5',
 'CAZON_cluster_4_moe_model.h5',
 'SARDINA_cluster_4_moe_model.h5',
 'RAYA Y SIMILARES_cluster_2_moe_model.h5',
 'PULPO_cluster_2_moe_model.h5',
 'CAMARON_cluster_2_moe_model.h5']

In [8]:
# Crear los modelos faltantes si hay suficientes datos
for model_name in missing_models:
    species_name, cluster_label = model_name.replace('_moe_model.h5', '').split('_cluster_')
    cluster_label = int(cluster_label)
    
    # Filtrar los datos para la combinación específica de especie y clúster
    filtered_data = data[(data['species'] == species_name) & (data['Cluster_Label'] == cluster_label)].dropna()
    
    if len(filtered_data) > 12:

        if len(filtered_data) >= look_back:
            filtered_data = filtered_data.sort_values('date')
            try:
                print(f'Training model for species: {species_name}, cluster: {cluster_label}')
                model, scaler = train_and_save_moe_model(filtered_data, species_name, cluster_label, look_back)

                # Verificar cuál es el último año de la serie de tiempo
                last_year = filtered_data['date'].dt.year.max()
                
                # Filtrar los datos del último año
                test_data_last_year = filtered_data[filtered_data['date'].dt.year == last_year]
                if len(test_data_last_year) >= look_back:
                    predictions, lower_bound, upper_bound = predict_with_confidence_intervals(model, scaler, test_data_last_year, look_back)
                    real_values = test_data_last_year['landed_w_kg'].values[look_back:]
                    
                    # Crear un DataFrame para los resultados
                    results_df = pd.DataFrame({
                        'date': test_data_last_year['date'].values[look_back:],
                        'real_values': real_values,
                        'predictions': predictions,
                        'lower_bound': lower_bound,
                        'upper_bound': upper_bound
                    })
                    
                    # Guardar los resultados
                    results_directory = 'resultados_moe'
                    os.makedirs(results_directory, exist_ok=True)
                    results_path = os.path.join(results_directory, f'{species_name}_cluster_{str(cluster_label)}_predictions.csv')
                    results_df.to_csv(results_path, index=False)
                    
                    # Agrupar por mes y sumar los valores
                    monthly_totals = results_df.set_index('date').resample('ME').sum()
                    
                    # Calcular MSE y R2 para los totales mensuales
                    mse = mean_squared_error(monthly_totals['real_values'], monthly_totals['predictions'])
                    r2 = r2_score(monthly_totals['real_values'], monthly_totals['predictions'])
                    print(f'MSE Mensual para {species_name} en clúster {cluster_label}: {mse}, R2 Mensual: {r2}')
                    
                    # Crear figura
                    plt.figure(figsize=(10, 6))
                    plt.plot(monthly_totals.index, monthly_totals['real_values'], color='blue', label='Real')
                    plt.plot(monthly_totals.index, monthly_totals['predictions'], color='red', linestyle='--', label='Predicho')
                    plt.fill_between(monthly_totals.index, monthly_totals['lower_bound'], monthly_totals['upper_bound'], color='gray', alpha=0.2, label='Intervalo de Confianza 95%')
                    plt.title(f'Totales Mensuales de {species_name} en clúster {cluster_label} para el Último Año')
                    plt.xlabel('Fecha')
                    plt.ylabel('Peso Desembarcado (kg)')
                    plt.legend()
                    plot_path = os.path.join(results_directory, f'{species_name}_cluster_{cluster_label}_plot.png')
                    plt.savefig(plot_path)
                    plt.close()
                else:
                    print(f'No se tienen datos suficientes para {species_name} en clúster {cluster_label} en el último año')
            except Exception as e:
                print(f'Error entrenando el modelo para especie: {species_name}, clúster: {cluster_label}. Error: {e}')

Training model for species: MOJARRA, cluster: 4
Epoch 1/50
38/38 [==============================] - 5s 32ms/step - loss: 0.0557 - val_loss: 0.1102
Epoch 2/50
38/38 [==============================] - 0s 6ms/step - loss: 0.0397 - val_loss: 0.1017
Epoch 3/50
38/38 [==============================] - 0s 6ms/step - loss: 0.0399 - val_loss: 0.1677
Epoch 4/50
38/38 [==============================] - 0s 6ms/step - loss: 0.0299 - val_loss: 0.1350
Epoch 5/50
38/38 [==============================] - 0s 6ms/step - loss: 0.0251 - val_loss: 0.1247
Epoch 6/50
38/38 [==============================] - 0s 5ms/step - loss: 0.0269 - val_loss: 0.1345
Epoch 7/50
38/38 [==============================] - 0s 6ms/step - loss: 0.0211 - val_loss: 0.1832
Epoch 8/50
38/38 [==============================] - 0s 6ms/step - loss: 0.0219 - val_loss: 0.1646
Epoch 9/50
38/38 [==============================] - 0s 5ms/step - loss: 0.0211 - val_loss: 0.1371
Epoch 10/50
38/38 [==============================] - 0s 6ms/step - lo

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
10/10 [==============================] - 5s 87ms/step - loss: 0.4117 - val_loss: 0.0165
Epoch 2/50
10/10 [==============================] - 0s 9ms/step - loss: 0.2640 - val_loss: 0.0059
Epoch 3/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1755 - val_loss: 0.0374
Epoch 4/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1613 - val_loss: 0.0776
Epoch 5/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1444 - val_loss: 0.0722
Epoch 6/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1387 - val_loss: 0.0624
Epoch 7/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1171 - val_loss: 0.0552
Epoch 8/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1173 - val_loss: 0.0487
Epoch 9/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1110 - val_loss: 0.0537
Epoch 10/50
10/10 [==============================] - 0s 8ms/step - loss: 0.1023 - val_loss: 0.0594
Epoch 11/50
10/10 

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
18/18 [==============================] - 5s 49ms/step - loss: 0.0610 - val_loss: 0.0272
Epoch 2/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0478 - val_loss: 0.0249
Epoch 3/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0373 - val_loss: 0.0268
Epoch 4/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0363 - val_loss: 0.0272
Epoch 5/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0327 - val_loss: 0.0246
Epoch 6/50
18/18 [==============================] - 0s 6ms/step - loss: 0.0326 - val_loss: 0.0272
Epoch 7/50
18/18 [==============================] - 0s 6ms/step - loss: 0.0273 - val_loss: 0.0304
Epoch 8/50
18/18 [==============================] - 0s 6ms/step - loss: 0.0250 - val_loss: 0.0279
Epoch 9/50
18/18 [==============================] - 0s 6ms/step - loss: 0.0223 - val_loss: 0.0302
Epoch 10/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0240 - val_loss: 0.0284
Epoch 11/50
18/18 

...

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
28/28 [==============================] - 5s 32ms/step - loss: 0.0578 - val_loss: 0.0412
Epoch 2/50
28/28 [==============================] - 0s 7ms/step - loss: 0.0470 - val_loss: 0.0200
Epoch 3/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0462 - val_loss: 0.0207
Epoch 4/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0399 - val_loss: 0.0197
Epoch 5/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0367 - val_loss: 0.0221
Epoch 6/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0383 - val_loss: 0.0149
Epoch 7/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0354 - val_loss: 0.0238
Epoch 8/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0280 - val_loss: 0.0130
Epoch 9/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0273 - val_loss: 0.0078
Epoch 10/50
28/28 [==============================] - 0s 6ms/step - loss: 0.0258 - val_loss: 0.0053
Epoch 11/50
28/28 

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 21ms/step
MSE Mensual para SARDINA en clúster 4: 1650952.1901642508, R2 Mensual: 0.8287546153301288
Training model for species: CAMARON, cluster: 2
Epoch 1/50
5/5 [==============================] - 6s 192ms/step - loss: 0.1757 - val_loss: 0.0114
Epoch 2/50
5/5 [==============================] - 0s 14ms/step - loss: 0.1472 - val_loss: 0.0046
Epoch 3/50
5/5 [==============================] - 0s 14ms/step - loss: 0.1275 - val_loss: 0.0024
Epoch 4/50
5/5 [==============================] - 0s 12ms/step - loss: 0.1164 - val_loss: 0.0040
Epoch 5/50
5/5 [==============================] - 0s 12ms/step - loss: 0.1012 - val_loss: 0.0100
Epoch 6/50
5/5 [==============================] - 0s 12ms/step - loss: 0.0985 - val_loss: 0.0242
Epoch 7/50
5/5 [==============================] - 0s 12ms/step - loss: 0.0829 - val_loss: 0.0328
Epoch 8/50
5/5 [==============================] - 0s 12ms/step - loss: 0.0864 - val_loss: 0.0580
Epoch 9/50
5/5 [=================

/opt/conda/envs/cedo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
